# Game in Python

In [3]:
import cv2
import numpy as np
import pyautogui
import random
import time
import mediapipe as mp

# Initialize variables
max_score = 20
min_score = -20
current_score = 0
circle_radius = 30
circle_speed = 2
circles = []
colors = [(0, 0, 255), (0, 255, 0), (0, 255, 255)]  # Red, Green, Yellow

# Initialize PyAutoGUI
pyautogui.FAILSAFE = False

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Object properties
object_radius = 20
object_color = (255, 0, 0)  # Object color (red)
object_x, object_y = 320, 480  # Initial object position

# Game state
game_over = False

# Time interval for generating a new group of circles (in seconds)
circle_generation_interval = 3
last_circle_generation_time = time.time()

# Create a function to draw circles
def draw_circles(frame):
    global circles

    # Update existing circles' positions
    new_circles = []
    for circle in circles:
        x, y, color = circle
        y += circle_speed
        if y < frame.shape[0]:
            cv2.circle(frame, (x, y), circle_radius, color, -1)
            new_circles.append((x, y, color))
        else:
            # Circle reached the bottom, remove it
            new_circles.append(circle)

    circles = new_circles

    # Check if it's time to generate a new group of circles
    if time.time() - last_circle_generation_time > circle_generation_interval:
        generate_new_circles(frame)

# Create a function to generate a new group of circles
def generate_new_circles(frame):
    global circles, last_circle_generation_time

    # Add new circles
    for _ in range(5):
        x = random.randint(circle_radius, frame.shape[1] - circle_radius)
        color = random.choice(colors)
        circles.append((x, -circle_radius, color))

    last_circle_generation_time = time.time()

# Create a function to check if the object is over a circle
def check_object_position(frame):
    global current_score, game_over

    for i, (circle_x, circle_y, color) in enumerate(circles):
        distance = np.sqrt((object_x - circle_x) ** 2 + (object_y - circle_y) ** 2)
        if distance < circle_radius + object_radius:
            if color == (0, 255, 0):
                current_score += 5
            elif color == (0, 0, 255):
                current_score -= 2
            elif color == (0, 255, 255):
                current_score -= 3
            del circles[i]
            break

    # Check for winning or losing conditions
    if current_score >= max_score:
        game_over = True
        cv2.putText(frame, "You won!", (220, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    elif current_score <= min_score:
        game_over = True
        cv2.putText(frame, "You lose!", (220, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

# Create a main game loop
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    if not ret:
        break

    if not game_over:
        # Detect hands and draw landmarks
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Get the position of the object based on hand landmarks
                object_x = int(landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * frame.shape[1])
                object_y = int(landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * frame.shape[0])

        # Draw circles
        draw_circles(frame)

        # Check if the object is over a circle and update the score
        check_object_position(frame)

        # Draw the object on the frame
        cv2.circle(frame, (object_x, object_y), object_radius, object_color, -1)

    # Display current score
    cv2.putText(frame, f"Score: {current_score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow("Game", frame)

    # Quit the game if 'q' is pressed or the game is over
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q') or game_over:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
